# `LaPDXYExclusion` Overview

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import xarray as xr

from matplotlib.patches import Polygon

sys.executable;

In [ ]:
try:
    from bapsf_motion.motion_builder.exclusions import (
        CircularExclusion,
        DividerExclusion,
        GovernExclusion,
        LaPDXYExclusion,
    )
except ModuleNotFoundError:
    from pathlib import Path

    HERE = Path().cwd()
    BAPSF_MOTION = (HERE / ".." / ".." / ".." ).resolve()
    sys.path.append(str(BAPSF_MOTION))
    
    from bapsf_motion.motion_builder.exclusions import (
        CircularExclusion,
        DividerExclusion,
        GovernExclusion,
        LaPDXYExclusion,
    )
    

In [ ]:
plt.rcParams.update(
    {
        # "figure.figsize": [12, 0.56 * 12],
        "figure.figsize": [10, 0.8 * 10],
        "font.size": 16,
    }
)

## Usage

Direct usage should never be needed, since the `MotionBuilder` will handle this given the correct configuration is given to `MotionBuilder`.  The appropriate TOML or dictionary like configurations can be found in the documentation for `LaPDXYExclusion`.

### Is a `GovernExclusion`

`LaPDXYExclusion` is a subclass of `GovernExclusion`.  This means the mask generated by `LaPDXYExclusion` will examine the existing global mask to generate its own mask, and that generated mask will replace the global mask.  As a result, there should only be one `GovernExclusion` used wheneve constructing a motion space.

In [ ]:
issubclass(LaPDXYExclusion, GovernExclusion)

### Is a Compound Exclusion

This means `LaPDXYExclusion` leverages other exclusion classes to generate its exclusion layer.  These exclusions can be view with the `composed_exclusions` attribute, which you can see in the next section.

### Defining a `LaPDXYExclusion`

Create an initial global mask.

In [ ]:
size = 111
side = np.linspace(-55, 55, num=size)
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

The default configuration parameters for `LaPDXYExclusion` define a probe drive beting deployed on the East side of the LaPD.

In [ ]:
ex = LaPDXYExclusion(ds)

ds.mask.plot(x="x", y="y");
plt.scatter(
    ex.insertion_point[0],
    ex.insertion_point[1],
    marker="+",
    color="black",
    s=8**2,
)
axis = plt.gca()
axis.set_xlim(-60, 60)
axis.set_ylim(-60, 60)

The configuration dictionary for this exclusion looks like...

In [ ]:
ex.config

You can see the exclusion is composed of 4 other exclusions, 1x `Shadow2DExclusion`, 1x `CircularExclusion`, and 3x `DividerExclusions`.

In [ ]:
ex.composed_exclusions

Let's make a slightly more complicated LaPD exlusion layer.  In this "complicated" scenario let's assume the probe drive is being deployed on the West side of the LaPD and there is a circular obstruction at the center of the chamber.

In [ ]:
# Reset the motion space
size = 111
side = np.linspace(-55, 55, num=size)
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

# Add circular obstruction at the center
ex_1 = CircularExclusion(ds, radius=5, center=(0,0), exclude="inside")

# Define West side lapd XY exclusion
ex = LaPDXYExclusion(ds, port_location="W")

# plot mask
ds.mask.plot(x="x", y="y");
plt.scatter(
    ex.insertion_point[0],
    ex.insertion_point[1],
    marker="+",
    color="black",
    s=8**2,
)
axis = plt.gca()
axis.set_xlim(-60, 60)
axis.set_ylim(-60, 60)

Now you can see the component `Shadow2DExclusion` is casting a shadow from the center circular exclusion, so the probe can not be moved to that location.

The confiruation for this setup looks like...

In [ ]:
ex.config